# Evaluate heuristics with different values of k to compute k-shortest paths
Evaluate heuristics on different topologies for a range of k.
Can be evaluated for fixed length episodes or episodes that end on first blocking event.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
# Run the train script for each value of k
root = "/home/uceedoh"
data_directory = f"{root}/git/XLRON/data"
data_directory = f"{root}/git/XLRON/data/heuristic_benchmarks"
script_path = f"{root}/git/XLRON/xlron/train/train.py"
modulations_csv_filepath = f"{root}/git/XLRON/examples/modulations.csv"
def check_file(file_path):
    if not os.path.exists(file_path):
        return False
    try:
        with open(file_path, 'r') as file:
            return sum(1 for _ in file) > 2
    except IOError:
        return False

In [2]:
import jax
jax.devices()

In [3]:
env_type = "rmsa"
topologies = ["nsfnet_deeprmsa", "cost239_deeprmsa"]
heuristics = ['ksp_ff', 'ff_ksp', "ksp_bf", "bf_ksp", "kme_ff", "kmc_ff", "kmf_ff", "kca_ff"]
k_range = range(1, 11)
load_range = [50, 100, 150, 200, 250, 300, 400, 500, 600, 800]
# Choice of traffic load - should be sufficient to cover wide range of blocking probs from <1% to 10%

commands = []
for topology in topologies:
    for heuristic in heuristics:
        for k in k_range:
            for load in load_range:
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}_k{k}_{load}.csv"
                if check_file(output_file):
                    print(f'Skipping file {output_file}')
                    pass
                else:
                    commands.append(f"python3 {script_path} --VISIBLE_DEVICES=3 --link_resources=100 --max_requests=1000 --max_timesteps=1000 --TOTAL_TIMESTEPS 10000 --NUM_ENVS 1 --NUM_SEEDS 100 --mean_service_holding_time=25 --ENV_WARMUP_STEPS 5000 --continuous_operation --noPLOTTING --ACTION_MASKING --EVAL_HEURISTIC --k {k} --env_type={env_type} --topology_name={topology} --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file} --load={load} --modulations_csv_filepath {modulations_csv_filepath}")

print(f"Total commands to run: {len(commands)}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Commands left: {len(commands) - i}")
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [4]:
env_type = "rwa_lightpath_reuse"
num_seeds = 100
topologies = ["cost239", "nsfnet"]
heuristics = ['ksp_ff', 'ff_ksp', 'ksp_mu']#, 'ksp_mu_nonrel', 'ksp_mu_unique', 'mu_ksp', 'mu_ksp_nonrel', 'mu_ksp_unique']
num_requests_nsfnet = [1e4, 1e4, 1.5e4, 2e4, 2.5e4]
num_requests_cost239 = [2e4, 2e4, 2.5e4, 3e4, 3.5e4]
k_range = range(1, 11)

commands = []
for topology in topologies:
    num_requests_list = num_requests_nsfnet if topology=="nsfnet" else num_requests_cost239
    for heuristic in heuristics:
        for i, num_requests in enumerate(num_requests_list):
            first_blocking = (i == 0)
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}_k{k}_{num_requests:.0f}{'_firstblocking' if first_blocking else ''}.csv"
                if check_file(output_file):
                    print(f'Skipping file {output_file}')
                    pass
                commands.append(f"python3 ../xlron/train/train.py --VISIBLE_DEVICES=3 --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests={num_requests} --max_timesteps={num_requests} --values_bw=100 --TOTAL_TIMESTEPS {int(num_requests)} --NUM_ENVS 1 --NUM_SEEDS {num_seeds} --ACTION_MASKING --incremental_loading {'--end_first_blocking' if first_blocking else ''} --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file}")

print(f"Total commands to run: {len(commands)}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Commands left: {len(commands) - i}")
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [7]:
# Read each of the generated files, get the mean accepted services (or metric of choice) and plot

for end_first_blocking in [False, True]:
    for topology in ["cost239", "nsfnet"]:
        accepted_services = []
        labels = []
        for heuristic in [
            'ksp_ff', 
            'ff_ksp', 
            #'ksp_mu', 
            #'mu_ksp', 
            "ksp_mu_alt", 
            "mu_ksp_alt"]:
            mean_accepted_services = []
            std_accepted_services = []
            for k in range (1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_firstblocking' if end_first_blocking else ''}.csv"
                df = pd.read_csv(output_file)
                accepted_services.append(df['accepted_services'])
                mean_accepted_services.append(df['accepted_services'].mean())
                std_accepted_services.append(df['accepted_services'].std())
                print(f"Mean accepted services for {heuristic}{k}: {df['accepted_services'].mean()}")
                labels.append(f"{heuristic}{k}")
            plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
            plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel("Mean accepted services")
        plt.legend()
        plt.grid()
        plt.title(f"{topology}-{'first blocking' if end_first_blocking else 'fixed length'}")
        plt.show()
        # Plot as a boxplot
        # plt.boxplot(accepted_services, labels=labels)
        # plt.xlabel("k")
        # plt.ylabel("Mean accepted services")
        # plt.legend()
        # plt.show()

In [ ]:
# Same but for first blocking
commands = []
for topology in ["cost239", "nsfnet"]:
    for num_requests in [1e4, 2e4, 3e4, 4e4]:
        for heuristic in ['ksp_ff', 'ff_ksp', 'ksp_mu', 'mu_ksp', "ksp_mu_alt", "mu_ksp_alt"]:
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{str(int(num_requests))}.csv"
                commands.append(f"python3 ../xlron/train/train.py --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests={num_requests} --max_timesteps={num_requests} --values_bw=100 --TOTAL_TIMESTEPS 120000 --NUM_ENVS 1 --NUM_SEEDS 1 --ACTION_MASKING --incremental_loading  --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file}")
            
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [9]:

for topology in ["cost239", "nsfnet"]:
    for num_requests in [1e4, 2e4, 3e4, 4e4]:
        accepted_services = []
        labels = []
        for heuristic in [
            'ksp_ff', 
            'ff_ksp', 
            'ksp_mu', 
            'mu_ksp', 
            "ksp_mu_alt", 
            "mu_ksp_alt"]:
            mean_accepted_services = []
            std_accepted_services = []
            for k in range (1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{str(int(num_requests))}.csv"
                df = pd.read_csv(output_file)
                accepted_services.append(df['accepted_services'])
                mean_accepted_services.append(df['accepted_services'].mean())
                std_accepted_services.append(df['accepted_services'].std())
                print(f"Mean accepted services for {heuristic}{k}: {df['accepted_services'].mean()}")
                labels.append(f"{heuristic}{k}")
            plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
            plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel("Mean accepted services")
        plt.legend()
        plt.grid()
        plt.title(f"{topology}-{num_requests}")
        plt.show()

In [18]:
# Same but for dynamic RMSA environment
env_type = "rmsa"
commands = []
for topology in ["nsfnet_deeprmsa"]:
    for weight in [True, False]:
        for heuristic in ['ksp_ff', 'ff_ksp', 'kmf_ff', 'kmc_ff']:#, 'ksp_mu', 'mu_ksp']:
            for k in range(1, 11):
                output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_weighted' if weight else ''}_deeprmsa.csv"
                commands.append(f"python3 ../xlron/train/train.py --k {k} --env_type={env_type} --topology_name={topology} --link_resources=100 --max_requests=1e3 --max_timesteps=1e3 --load=250 --mean_service_holding_time=25 --TOTAL_TIMESTEPS 120000 --NUM_ENVS 1 --NUM_SEEDS 1 --ACTION_MASKING  --EVAL_HEURISTIC --path_heuristic {heuristic} --DATA_OUTPUT_FILE {output_file} --truncate_holding_time --ENV_WARMUP_STEPS 3000 --continuous_operation {'--weight weight' if weight else ''}")
# Loop through the commands and run each one
for i, cmd in enumerate(commands):
    print(f"Running command {i+1}: {cmd}")
    !{cmd}

In [20]:
env_type = "rmsa"
commands = []
for metric in ["service_blocking_probability", "bitrate_blocking_probability"]:
    for topology in ["nsfnet_deeprmsa"]:
        for weight in [True, False]:
            accepted_services = []
            labels = []
            for heuristic in ['ksp_ff', 'ff_ksp', 'kmc_ff']:#, "kmf_ff"]:#, 'mu_ksp', 'ksp_mu']: 
                mean_accepted_services = []
                std_accepted_services = []
                for k in range (1, 11):
                    output_file = f"{data_directory}/kpaths/{env_type}/{topology}/{heuristic}{k}{'_weighted' if weight else ''}_deeprmsa.csv"
                    df = pd.read_csv(output_file)
                    accepted_services.append(df[metric])
                    mean_accepted_services.append(df[metric].mean())
                    std_accepted_services.append(df[metric].std())
                    print(f"Mean {metric} for {heuristic}{k}: {df[metric].mean()}")
                    labels.append(f"{heuristic}{k}")
                plt.plot(range(1, 11), mean_accepted_services, label=heuristic)
                plt.fill_between(range(1, 11), np.array(mean_accepted_services) - np.array(std_accepted_services), np.array(mean_accepted_services) + np.array(std_accepted_services), alpha=0.2)
        plt.xlabel("k")
        plt.xticks(range(1, 11))
        plt.ylabel(f"Mean {metric}")
        plt.legend()
        plt.grid()
        plt.show()

In [4]:
# To find the effect of truncating the holding time ot be less than 2*mean
import numpy as np

def deeprmsa_sample(mean):
    holding_time = np.random.exponential(mean)
    while holding_time > 2*mean:
        holding_time = np.random.exponential(mean)
    return holding_time

# plot the distribution of holding times
mean = 25
deeprmsa = [deeprmsa_sample(mean) for i in range(100000)]
plt.hist(deeprmsa, bins=100)
plt.show()

In [ ]:
ratios = []
for mean_holding_time in [5,10,15,20,25,30]:
    hts = []
    for i in range(10000000):
        holding_time = deeprmsa_sample(mean_holding_time)
        hts.append(holding_time)
    real_mean_ht = np.mean(hts)
    ratio = real_mean_ht/mean_holding_time
    ratios.append(ratio)
    print(f"Real mean: {real_mean_ht}")
    print(f"Ration of real to expected: {real_mean_ht/mean_holding_time}")
real_mean = np.mean(ratios)
real_std = np.sqrt(np.sum(((np.array(ratios)-real_mean)**2)/(len(ratios)-1)))
print(real_mean)
print(real_std)